MOCK-UP OF GROUP 2 SUPERVISED ML MODEL: Logistic REGRESSION

In [1]:
# Import dependencies
import pandas as pd 
from pathlib import Path 
import matplotlib.pyplot as plt 

In [2]:
# # Load tornado data CSV as a Pandas DataFrame and preview the DataFrame
# file_path = Path('../Cleaned_Data/2008-2020_tornadoes_EF_cleaned.csv')
# df = pd.read_csv(file_path, index_col=0)
# df.head(25)

In [4]:
# Apache Spark Set Up

import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

'apt-get' is not recognized as an internal or external command,
operable program or batch file.
The system cannot find the path specified.
'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open '$SPARK_VERSION-bin-hadoop2.7.tgz'


In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Tornadoes").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://group-2-project-tornadoes.s3.amazonaws.com/2008-2020_tornadoes_EF_cleaned_db.csv"
spark.sparkContext.addFile(url)
tornado = spark.read.csv(SparkFiles.get("2008-2020_tornadoes_EF_cleaned_db.csv"), sep=",", header=True)

# Show DataFrame
tornado.show()

In [ ]:
# Convert spark object to pandas dataframe
tornado_df = tornado.toPandas()
tornado_df

In [ ]:
# Check data types
tornado_df.dtypes

In [ ]:
# Convert timestamp to datetime
tornado_df['Timestamp']= pd.to_datetime(tornado_df['Timestamp'])

# Convert all strings to numeric
cols=[i for i in tornado_df.columns if i not in ["Timestamp"]]
for col in cols:
    tornado_df[col]=pd.to_numeric(tornado_df[col])

# check dtypes
tornado_df.dtypes

In [ ]:
# Review columns
df.columns

In [ ]:
# Visually inspect relationship between tornado magnitude ('EF' column) and 'Loss' - positive trend
plt.scatter(df.EF, df.Loss) 
plt.xlabel('Tornado Magnitude') 
plt.ylabel('Loss') 
plt.show()

In [ ]:
# Visually inspect relationship between tornado magnitude ('EF' column) and 'Length' - positive trend
plt.scatter(df.EF, df.Length) 
plt.xlabel('Tornado Magnitude') 
plt.ylabel('Length') 
plt.show()

In [ ]:
# Visually inspect relationship between tornado magnitude ('EF' column) and 'Width' - positive trend
plt.scatter(df.EF, df.Width) 
plt.xlabel('Tornado Magnitude') 
plt.ylabel('Width') 
plt.show()

In [ ]:
# Format data to meet the requirements of the Scikit-learn library: 
# Will add Loss later
feature_names = ["Length", "Width"]
X = df[feature_names]


In [ ]:
# Assign the target variable - EF
y = df.EF.values.reshape(-1, 1)
y

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
## Logistic Regression

# Our data is not linear - Reject model

from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train_scaled, y_train)

print('Accuracy of logistic regression on training', logreg.score(X_train_scaled, y_train))
print('Accuracy of logistic regression on testing', logreg.score(X_test_scaled, y_test))